# 머신 러닝 교과서 - 파이토치편

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/ml-with-pytorch/blob/main/ch13/ch13_part4_ignite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

## 패키지 버전 체크

check_packages.py 스크립트에서 로드하기 위해 폴더를 추가합니다:

In [1]:
import sys

# 코랩의 경우 깃허브 저장소로부터 python_environment_check.py를 다운로드 합니다.
if 'google.colab' in sys.modules:
    !wget https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/python_environment_check.py
else:
    sys.path.insert(0, '..')

--2023-08-11 02:37:30--  https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/python_environment_check.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1629 (1.6K) [text/plain]
Saving to: ‘python_environment_check.py’

python_environment_ 100%[===================>]   1.59K  --.-KB/s    in 0s      

2023-08-11 02:37:30 (30.5 MB/s) - ‘python_environment_check.py’ saved [1629/1629]



권장 패키지 버전을 확인하세요:

In [2]:
from python_environment_check import check_packages


d = {
    'numpy': '1.21.2',
    'matplotlib': '3.4.3',
    'sklearn': '1.0',
}
check_packages(d)

[OK] Your Python version is 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
[OK] numpy 1.23.5
[OK] matplotlib 3.7.1
[OK] sklearn 1.2.2


# 13장 - 파이토치 구조 자세히 알아보기 (파트 4)

**이 섹션의 초안을 작성하고 도움을 주신 Victor Fomin에게 큰 감사와 공로를 돌립니다!**

## 파이토치 이그나이트 소개 (온라인 보너스 콘텐츠)

이 섹션에서는 파이토치에서 유연하고 투명하게 신경망을 훈련하고 평가하는 데 도움이 되는 파이토치 에코시스템의 라이브러리인 파이토치 이그나이트(PyTorch-Ignite)를 살펴보겠습니다.

**파이토치 이그나이트를 사용한 프로젝트**


파이토치 이그나이트를 사용하는 연구 논문, 블로그, 튜토리얼, 프로젝트가 많습니다. 그 중 유명한 프로젝트는 다음과 같습니다.
- Medical Open Network for AI (MONAI) (https://monai.io)
- Conversational AI with Transfer Learning (https://github.com/huggingface/transfer-learning-conv-ai)

파이토치 이그나이트를 사용하는 더 많은 프로젝트는 다음 링크를 확인하세요: https://github.com/pytorch/ignite#projects-using-ignite

---

이전에 살펴본 것처럼, 파이토치 훈련 코드에는 일반적으로 두 개의 중첩된 for 루프가 포함됩니다. 하나는 에포크에 대해 반복하고 다른 하나는 데이터셋 배치에 대해 반복합니다. 또한 훈련 및 검증 세트에서 모델을 평가하여 훈련 중 성능을 추적합니다. 일반적으로 훈련 체크포인트를 생성하고(갑작스럽게 훈련이 실패할 경우 다시 시작하기 위해), 최상의 모델을 저장하고, 추적 시스템을 사용하여 손실, 예측 등을 시각화하는 등 입니다. 이러한 작업은 파이토치 이그나이트가 파이토치와 같은 유연성을 유지하면서 쉽게 처리할 수 있는 종류의 작업입니다. 이러한 의미에서 파이토치 이그나이트는 모범 사례를 촉진시키며 모델 훈련 프로세스를 간소화하는 것을 목표로 합니다.

파이토치 이그나이트는 다음과 같은 기능을 제공합니다.
- 매우 간단한 엔진 및 이벤트 시스템(훈련 루프 추상화)
- 모델을 쉽게 평가할 수 있는 지표를 기본적으로 제공
- 훈련 파이프라인을 구성하고, 모델을 저장하고, 매개변수와 측정 지표를 기록하는 내장 핸들러
- 분산 훈련 지원

파이토치 이그나이트 사용의 추가 이점은 다음과 같습니다.
- 최대한의 제어와 단순성을 보장하면서 순수한 파이토치 보다 적은 코드량
- 더 모듈화된 코드

이 절에서는 MNIST 데이터셋에 대한 분류기를 다시 만들고 훈련해 보겠습니다.

---

**파이토치 이그나이트 설치**

이 노트북의 코드는 파이토치 이그나이트 버전 0.4.6을 기반으로 합니다. 파이토치 이그나이트는 pip 또는 conda를 통해 설치할 수 있습니다. 예를 들어, pip로 파이토치 이그나이트를 설치하는 명령은 다음과 같습니다:

    pip install pytorch-ignite

아래는 conda를 통해 파이토치 이그나이트를 설치하는 명령어입니다:

    conda install ignite -c pytorch

파이토치 이그나이트 설치에 대한 최신 정보는 공식 문서(https://pytorch.org/ignite/#installation)를 참조하시기 바랍니다.

In [5]:
!pip install pytorch-ignite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 3.9 MB/s eta 0:00:00


### 파이토치 모델 준비하기

먼저, *프로젝트 2 - MNIST 손글씨 숫자 분류* 섹션의 1, 2, 3단계를 반복합니다. 모델, 훈련 및 검증 데이터셋, 옵티마이저, 손실 함수를 정의합니다:

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision.datasets import MNIST
from torchvision import transforms


image_path = './'
torch.manual_seed(1)

transform = transforms.Compose([
    transforms.ToTensor()
])


mnist_train_dataset = MNIST(
    root=image_path,
    train=True,
    transform=transform,
    download=True
)

mnist_val_dataset = MNIST(
    root=image_path,
    train=False,
    transform=transform,
    download=False
)

batch_size = 64
train_loader = DataLoader(
    mnist_train_dataset, batch_size, shuffle=True
)

val_loader = DataLoader(
    mnist_val_dataset, batch_size, shuffle=False
)


def get_model(image_shape=(1, 28, 28), hidden_units=(32, 16)):
    input_size = image_shape[0] * image_shape[1] * image_shape[2]
    all_layers = [nn.Flatten()]
    for hidden_unit in hidden_units:
        layer = nn.Linear(input_size, hidden_unit)
        all_layers.append(layer)
        all_layers.append(nn.ReLU())
        input_size = hidden_unit

    all_layers.append(nn.Linear(hidden_units[-1], 10))
    all_layers.append(nn.Softmax(dim=1))
    model = nn.Sequential(*all_layers)
    return model


device = "cuda" if torch.cuda.is_available() else "cpu"

model = get_model().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

100%|██████████| 9912422/9912422 [00:00<00:00, 92337734.33it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 31028610.10it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 25378859.17it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 7254580.64it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



여기서 보듯이 위의 코드는 이전에 소개한 파이토치 개념에만 의존하고 있습니다. 재사용 가능한 `get_model()` 함수를 정의하여 임의의 갯수의 은닉층을 가진 다층 퍼셉트론을 편리하게 생성할 수 있습니다. 은닉층 다음에는 ReLU 활성화가 이어집니다. 출력층 뒤에는 소프트맥스 함수가 이어집니다.

MNIST 데이터셋에는 사전에 정의된 검증 세트 분할이 없습니다. 단순화를 위해 테스트 데이터 세트를 검증 세트로 사용했습니다. 하지만 모델 선택에 검증 세트를 사용하면 모델 성능에 대한 편향된 추정치를 제공하게 됩니다.

### 파이토치 이그나이트로 훈련과 평가 엔진 설정하기

가장 중요한 부분의 설정이 끝나면 파이토치 이그나이트가 다른 모든 반복적인 코드를 처리합니다. 다음으로, 지도 학습 모델을 간편하게 훈련하기 위해 모델, 옵티마이저, 손실 함수를 `ignite.engine.create_supervised_trainer()` 함수에 전달하여 트레이너 엔진을 정의해야 합니다(https://pytorch.org/ignite/generated/ignite.engine.create_supervised_trainer.html). 또한, 파이토치 이그나이트의 기본 지표와 모델을 `ignite.engine.create_supervised_evaluator()` 함수에 전달하여 평가 엔진을 생성합니다(https://pytorch.org/ignite/generated/ignite.engine.create_supervised_evaluator.html#create-supervised-evaluator):

In [6]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss


trainer = create_supervised_trainer(
    model, optimizer, loss_fn, device=device
)

val_metrics = {
    "accuracy": Accuracy(),
    "loss": Loss(loss_fn)
}

evaluator = create_supervised_evaluator(
    model, metrics=val_metrics, device=device
)

`trainer`와 `evaluator` 객체는 모두 파이토치 이그나이트의 핵심 구성 요소 중 하나인 `Engine` 클래스(https://pytorch.org/ignite/generated/ignite.engine.engine.Engine.html#ignite.engine.engine.Engine)의 인스턴스입니다. 훈련 또는 검증 루프를 추상화한 것입니다.

### 로깅과 검증을 위한 이벤트 핸들러 만들기

모든 종류의 이벤트 핸들러를 추가하여 코드를 커스터마이징할 수 있습니다. `Engine`에 실행 중에 트리거되는 다양한 이벤트에 대한 핸들러를 추가할 수 있습니다. 이벤트가 트리거되면 추가된 핸들러(함수)가 실행됩니다. 로깅을 위해 모든 `log_interval` 반복이 끝날 때마다 실행될 함수를 추가합니다:

In [7]:
# 훈련 상태를 기록하기 전에 대기해야 하는 배치 수
log_interval = 100

@trainer.on(Events.ITERATION_COMPLETED(every=log_interval))
def log_training_loss():
    e = trainer.state.epoch
    max_e = trainer.state.max_epochs
    i = trainer.state.iteration
    batch_loss = trainer.state.output
    print(f"Epoch[{e}/{max_e}], Iter[{i}] Loss: {batch_loss:.2f}")

또는 데코레이터 없이 `add_event_handler()` 호출을 통해 핸들러 함수를 트레이너에 첨부할 수 있습니다(https://pytorch.org/ignite/generated/ignite.engine.engine.Engine.html#ignite.engine.engine.Engine.add_event_handler).

위에서 만든 훈련 상태 로깅을 위한 이벤트 핸들러를 만든 것과 유사하게, 각 에포크마다 검증 지표를 계산하기 위한 이벤트 핸들러를 만들 수 있습니다. 다음 코드를 통해 에포크가 완료되면 검증 세트 데이터 로더인 `val_loader`에서 `evaluator`를 실행할 것입니다:

In [8]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results():
    eval_state = evaluator.run(val_loader)
    metrics = eval_state.metrics
    e = trainer.state.epoch
    max_e = trainer.state.max_epochs
    acc = metrics['accuracy']
    avg_loss = metrics['loss']
    print(f"검증 결과 - 에포크[{e}/{max_e}] 평균 정확도: {acc:.2f} 평균 손실: {avg_loss:.2f}")

### 훈련 체크포인트를 설정하고 최상의 모델 저장하기

훈련 과정에서 트레이너, 모델, 옵티마이저 및 기타 관련 객체를 저장하는 것이 일반적인 관행입니다. 이렇게 하면 갑작스럽게 훈련이 중단되는 경우 체크포인트에서 모델 훈련을 재개할 수 있습니다. 여기서는 기본 제공되는 파이토치 이그나이터 핸들러를 사용하여 각 에포크에 대한 훈련 체크포인트를 설정하겠습니다:

In [9]:
from ignite.handlers import Checkpoint, DiskSaver

# 체크포인트에 다음을 저장합니다:
to_save = {"model": model, "optimizer": optimizer, "trainer": trainer}

# 로컬 디스크에 체크포인트를 저장합니다.
output_path = "./output"
save_handler = DiskSaver(dirname=output_path, require_empty=False)

# 핸들러 준비:
checkpoint_handler = Checkpoint(
    to_save, save_handler, filename_prefix="training")

# 핸들러를 트레이너에 추가하기
trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpoint_handler)

위의 코드를 통해 나중에 모델을 저장하고 로드할 수 있는 `Checkpoint` 객체(https://pytorch.org/ignite/generated/ignite.handlers.checkpoint.Checkpoint.html#ignite.handlers.checkpoint.Checkpoint)를 만들었습니다.

중단된 훈련 실행을 재개할 수 있도록 모델을 저장하는 것 외에도, 추론 단계에서 나중에 예측을 할 수 있도록 최적의 모델을 저장하는 것이 좋습니다. 그런 다음 12장의 훈련된 모델 저장 및 다시 로드하기 섹션에 설명된 대로 `torch.load`를 통해 저장된 모델을 로드할 수 있습니다.

일반적으로 가장 좋은 모델은 검증 지표의 값에 의해 결정됩니다. 여기서는 동일한 핸들러인 `Checkpoint`를 사용하여 가장 높은 검증 정확도에 따라 최적의 모델을 저장하겠습니다:

In [10]:
# 검증 정확도를 기준으로 최상의 모델을 저장합니다.
best_model_handler = Checkpoint(
    {"model": model},
    save_handler,
    filename_prefix="best",
    n_saved=1,
    score_name="accuracy",
    score_function=Checkpoint.get_default_score_fn("accuracy"),
)

evaluator.add_event_handler(Events.COMPLETED, best_model_handler)

### 실험 추적 시스템으로 텐서보드 설정하기

다양한 구성으로 훈련을 실행할 때, 실험 추적 시스템(예: 텐서보드)을 사용하여 매개변수와 측정 지표를 기록하고 실험을 비교하는 것이 일반적인 관행입니다. 여기서는 `TensorboardLogger`(https://pytorch.org/ignite/generated/ignite.contrib.handlers.tensorboard_logger.html#ignite.contrib.handlers.tensorboard_logger.TensorboardLogger)를 사용하여 트레이너의 손실 및 검증 지표를 기록하겠습니다:

In [11]:
from ignite.contrib.handlers import TensorboardLogger, global_step_from_engine


tb_logger = TensorboardLogger(log_dir=output_path)

# 100번 반복마다 트레이너의 손실을 출력하는 핸들러
tb_logger.attach_output_handler(
    trainer,
    event_name=Events.ITERATION_COMPLETED(every=100),
    tag="training",
    output_transform=lambda loss: {"batch_loss": loss},
)

# 에포크가 끝날 때마다 evaluator의 지표를 출력하는 핸들러
tb_logger.attach_output_handler(
    evaluator,
    event_name=Events.EPOCH_COMPLETED,
    tag="validation",
    metric_names="all",
    global_step_transform=global_step_from_engine(trainer),
)

### 파이토치 이그나이트 모델 훈련 코드 실행하기

이제 트레이너가 설정되어 실행할 준비가 되었습니다. `run()` 메서드를 통해 5개의 에포크에 대해 모델을 훈련해 보겠습니다:

In [12]:
trainer.run(train_loader, max_epochs=5)

Epoch[1/5], Iter[100] Loss: 1.82
Epoch[1/5], Iter[200] Loss: 1.63
Epoch[1/5], Iter[300] Loss: 1.65
Epoch[1/5], Iter[400] Loss: 1.62
Epoch[1/5], Iter[500] Loss: 1.56
Epoch[1/5], Iter[600] Loss: 1.62
Epoch[1/5], Iter[700] Loss: 1.57
Epoch[1/5], Iter[800] Loss: 1.53
Epoch[1/5], Iter[900] Loss: 1.52
검증 결과 - 에포크[1/5] 평균 정확도: 0.91 평균 손실: 1.56
Epoch[2/5], Iter[1000] Loss: 1.54
Epoch[2/5], Iter[1100] Loss: 1.55
Epoch[2/5], Iter[1200] Loss: 1.58
Epoch[2/5], Iter[1300] Loss: 1.53
Epoch[2/5], Iter[1400] Loss: 1.56
Epoch[2/5], Iter[1500] Loss: 1.55
Epoch[2/5], Iter[1600] Loss: 1.52
Epoch[2/5], Iter[1700] Loss: 1.55
Epoch[2/5], Iter[1800] Loss: 1.55
검증 결과 - 에포크[2/5] 평균 정확도: 0.93 평균 손실: 1.54
Epoch[3/5], Iter[1900] Loss: 1.50
Epoch[3/5], Iter[2000] Loss: 1.52
Epoch[3/5], Iter[2100] Loss: 1.53
Epoch[3/5], Iter[2200] Loss: 1.56
Epoch[3/5], Iter[2300] Loss: 1.53
Epoch[3/5], Iter[2400] Loss: 1.54
Epoch[3/5], Iter[2500] Loss: 1.54
Epoch[3/5], Iter[2600] Loss: 1.60
Epoch[3/5], Iter[2700] Loss: 1.58
Epoch[3

State:
	iteration: 4690
	epoch: 5
	epoch_length: 938
	max_epochs: 5
	output: 1.4836150407791138
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

`tensorboard --logdir ./output` 명령으로 텐서보드를 실행하여 브라우저에서 대시보드를 볼 수 있습니다:

In [14]:
from IPython.display import Image
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch13/figures/ignite-01.png')

---

**파이토치 이그나이트 더 자세히 알아 보기**

파이토치 이그나이트에 대해 자세히 알아보려면 공식 웹사이트(https://pytorch-ignite.ai)에서 튜토리얼과 사용 방법 가이드를 확인하세요.

무엇보다도 이 웹사이트에는 편리한 파이토치 이그나이트 코드 생성기 애플리케이션(https://code-generator.pytorch-ignite.ai/)도 포함되어 있어 모든 것을 처음부터 다시 작성하지 않고도 작업을 시작할 수 있습니다.

파이토치 이그나이트의 코드는 깃허브(https://github.com/pytorch/ignite)에 공개되어 있습니다. 이 프로젝트는 커뮤니티의 노력으로 이루어지며, 배경과 기술에 관계없이 누구나 기여하고 기여자 커뮤니티에 참여할 수 있습니다!

---